In [1]:
from string import punctuation
from nltk import word_tokenize
from nltk.corpus import stopwords
import sent2vec


expression_unified_ds = [line.strip().split("\t") for line in open("../unified_dataset/expression.tsv")][1:]
kinaseact_unified_ds = [line.strip().split("\t") for line in open("../unified_dataset/kinaseact.tsv")][1:]

stop_words = set(stopwords.words('english'))
def preprocess_sentence(text):
    text = text.replace('/', ' / ')
    text = text.replace('.-', ' .- ')
    text = text.replace('.', ' . ')
    text = text.replace('\'', ' \' ')
    text = text.lower()

    tokens = [token for token in word_tokenize(text) if token not in punctuation and token not in stop_words]

    return ' '.join(tokens)

sentences_only_expression_data = [preprocess_sentence(row[1]) for row in expression_unified_ds]
sentences_only_kinaseact_data = [preprocess_sentence(row[1]) for row in kinaseact_unified_ds]
bio_sent_vec_model_location = "../biosentvec/model.bin"

In [2]:
model_path = bio_sent_vec_model_location
biosentvec_model = sent2vec.Sent2vecModel()
try:
    biosentvec_model.load_model(model_path)
except Exception as e:
    print(e)
print('model successfully loaded')

model successfully loaded


In [3]:
corpus_expr = biosentvec_model.embed_sentences(sentences_only_expression_data)
corpus_kinaseact = biosentvec_model.embed_sentences(sentences_only_kinaseact_data)

In [4]:
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score
from sklearn.model_selection import RandomizedSearchCV, cross_validate, StratifiedKFold
from models import POSSIBLE_CLASSIFIERS


def train_classifier(X, y):

    best_score = 0
    best_classifier = None
    best_params = None
    best_classifier_name = ""

    stratified_k_folds = StratifiedKFold(n_splits=5)
    
    scoring = {'precision': make_scorer(precision_score),
           'recall': make_scorer(recall_score),
           'f1': make_scorer(f1_score)}

    best_results = {}
    best_index = 0
    # Iterate over classifiers and perform grid search
    for classifier_name, classifier_info in POSSIBLE_CLASSIFIERS.items():
        random_search = RandomizedSearchCV(estimator=classifier_info['model'], n_iter=100,
                                           param_distributions=classifier_info['params'], cv=stratified_k_folds,
                                           scoring=scoring, refit='f1',
                                           verbose=1, n_jobs=-1)
        random_search.fit(X, y)

        print(f"Finished training model and fitting best hyperparameters for {classifier_name}. F1 score: "
              f"{str(random_search.best_score_)}")

        if random_search.best_score_ > best_score:
            best_score = random_search.best_score_
            best_classifier = random_search.best_estimator_
            best_params = random_search.best_params_
            best_classifier_name = classifier_name
            best_results = random_search.cv_results_
            best_index = random_search.best_index_

    # Retrieve the average precision, recall, and F1 score
    average_precision = best_results['mean_test_precision'][best_index]
    std_precision = best_results['std_test_precision'][best_index]
    average_recall = best_results['mean_test_recall'][best_index]
    std_recall = best_results['std_test_recall'][best_index]
    average_f1 = best_results['mean_test_f1'][best_index]
    std_f1 = best_results['std_test_f1'][best_index]

    # Return the trained model and performance metrics
    return best_classifier, average_precision, average_recall, average_f1, best_classifier_name, best_params, std_precision, std_recall, std_f1


/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/xgboost/core.py:265: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc 2.28+) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(


## Task 1

### Expression

In [5]:
import joblib

y = [int(row[2]) for row in expression_unified_ds]
classifier, precision, recall, fscore, classifier_name, params, std_precision, std_recall, std_f1 = train_classifier(corpus_expr, y)
joblib.dump(classifier, '../classifiers/all_info_expression.joblib')
print(f'precision: {str(precision)} ± {str(std_precision)}')
print(f'recall: {str(recall)} ± {str(std_recall)}')
print(f'fscore: {str(fscore)} ± {str(std_f1)}')
print(f'selected model: {classifier_name}')
print(f'selected params {str(params)}')

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Finished training model and fitting best hyperparameters for LogisticRegression. F1 score: 0.7461735530186652
Fitting 5 folds for each of 100 candidates, totalling 500 fits


/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/scipy/optimize/_linesearch.py:306: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/valerio/workspace/caltech/sentence_classification/venv/l

Finished training model and fitting best hyperparameters for RandomForestClassifier. F1 score: 0.4096507500604335
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Finished training model and fitting best hyperparameters for GradientBoostingClassifier. F1 score: 0.6648959346430655
Fitting 5 folds for each of 100 candidates, totalling 500 fits


/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

Finished training model and fitting best hyperparameters for XGBClassifier. F1 score: 0.6708875822195505
Fitting 5 folds for each of 100 candidates, totalling 500 fits


...

Finished training model and fitting best hyperparameters for KNeighborsClassifier. F1 score: 0.7036636945564423
Fitting 5 folds for each of 100 candidates, totalling 500 fits


/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

Finished training model and fitting best hyperparameters for SGDClassifier. F1 score: 0.6726635952421762
Fitting 5 folds for each of 100 candidates, totalling 500 fits


/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

Finished training model and fitting best hyperparameters for Perceptron. F1 score: 0.6852952070646341
precision: 0.7180528913505339 ± 0.2584376774788825
recall: 0.9 ± 0.03127535391881504
fscore: 0.762343434084235 ± 0.19630355895655446
selected model: SVC
selected params {'C': 30.374612704859594, 'class_weight': 'balanced', 'coef0': 4.044782717167201, 'decision_function_shape': 'ovo', 'degree': 4, 'gamma': 0.001491478567445433, 'kernel': 'rbf', 'random_state': 15, 'shrinking': False, 'tol': 0.0031266056226033853}


### Kinaseact

In [6]:
y = [int(row[2]) for row in kinaseact_unified_ds]
classifier, precision, recall, fscore, classifier_name, params, std_precision, std_recall, std_f1 = train_classifier(corpus_kinaseact, y)
joblib.dump(classifier, '../classifiers/all_info_kinase.joblib')
print(f'precision: {str(precision)} ± {str(std_precision)}')
print(f'recall: {str(recall)} ± {str(std_recall)}')
print(f'fscore: {str(fscore)} ± {str(std_f1)}')
print(f'selected model: {classifier_name}')
print(f'selected params {str(params)}')

Fitting 5 folds for each of 100 candidates, totalling 500 fits


/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valerio/workspace/caltech/sentence_

Finished training model and fitting best hyperparameters for LogisticRegression. F1 score: 0.7184660854206848
Fitting 5 folds for each of 100 candidates, totalling 500 fits


/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

Finished training model and fitting best hyperparameters for RandomForestClassifier. F1 score: 0.4184932500949305
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Finished training model and fitting best hyperparameters for GradientBoostingClassifier. F1 score: 0.6024501102022914
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Finished training model and fitting best hyperparameters for XGBClassifier. F1 score: 0.6114527913943977
Fitting 5 folds for each of 100 candidates, totalling 500 fits


...

Finished training model and fitting best hyperparameters for KNeighborsClassifier. F1 score: 0.5790538070411496
Fitting 5 folds for each of 100 candidates, totalling 500 fits


/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

Finished training model and fitting best hyperparameters for SGDClassifier. F1 score: 0.6060522538041917
Fitting 5 folds for each of 100 candidates, totalling 500 fits


/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

Finished training model and fitting best hyperparameters for Perceptron. F1 score: 0.6040912132243401
precision: 0.6875724982863549 ± 0.25436621697945744
recall: 0.8783424077434967 ± 0.08962720596935922
fscore: 0.7263719477731312 ± 0.17756652784019716
selected model: SVC
selected params {'C': 10.20133067583881, 'class_weight': 'balanced', 'coef0': 4.1720610308143184, 'decision_function_shape': 'ovo', 'degree': 5, 'gamma': 0.0025218517032508108, 'kernel': 'rbf', 'random_state': 85, 'shrinking': True, 'tol': 0.008317467245924519}


## Task 2

### Expression

In [7]:
y = [int(row[3]) for row in expression_unified_ds]
classifier, precision, recall, fscore, classifier_name, params, std_precision, std_recall, std_f1 = train_classifier(corpus_expr, y)
joblib.dump(classifier, '../classifiers/curatable_expression.joblib')
print(f'precision: {str(precision)} ± {str(std_precision)}')
print(f'recall: {str(recall)} ± {str(std_recall)}')
print(f'fscore: {str(fscore)} ± {str(std_f1)}')
print(f'selected model: {classifier_name}')
print(f'selected params {str(params)}')

Fitting 5 folds for each of 100 candidates, totalling 500 fits


/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

Finished training model and fitting best hyperparameters for LogisticRegression. F1 score: 0.8209826141874554
Fitting 5 folds for each of 100 candidates, totalling 500 fits


/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

Finished training model and fitting best hyperparameters for RandomForestClassifier. F1 score: 0.6860880894826881
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Finished training model and fitting best hyperparameters for GradientBoostingClassifier. F1 score: 0.788314330731009
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Finished training model and fitting best hyperparameters for XGBClassifier. F1 score: 0.7847873824251754
Fitting 5 folds for each of 100 candidates, totalling 500 fits


...

Finished training model and fitting best hyperparameters for KNeighborsClassifier. F1 score: 0.7817382080464846
Fitting 5 folds for each of 100 candidates, totalling 500 fits


/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

Finished training model and fitting best hyperparameters for SGDClassifier. F1 score: 0.8108564169315391
Fitting 5 folds for each of 100 candidates, totalling 500 fits


/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

Finished training model and fitting best hyperparameters for Perceptron. F1 score: 0.7969579910443538
precision: 0.8444159505189586 ± 0.19511072318923517
recall: 0.843488253319714 ± 0.08665174123876136
fscore: 0.8221136127767512 ± 0.10739563750028534
selected model: MLPClassifier
selected params {'activation': 'relu', 'alpha': 0.004739790420737936, 'beta_1': 0.10145199114963827, 'beta_2': 0.0017098955783791273, 'epsilon': 0.0026560873272834917, 'hidden_layer_sizes': (500, 500), 'learning_rate': 'invscaling', 'learning_rate_init': 0.00015172001709703892, 'solver': 'adam'}


### Kinaseact

In [8]:
y = [int(row[3]) for row in kinaseact_unified_ds]
classifier, precision, recall, fscore, classifier_name, params, std_precision, std_recall, std_f1 = train_classifier(corpus_kinaseact, y)
joblib.dump(classifier, '../classifiers/curatable_kinase.joblib')
print(f'precision: {str(precision)} ± {str(std_precision)}')
print(f'recall: {str(recall)} ± {str(std_recall)}')
print(f'fscore: {str(fscore)} ± {str(std_f1)}')
print(f'selected model: {classifier_name}')
print(f'selected params {str(params)}')

Fitting 5 folds for each of 100 candidates, totalling 500 fits


/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

Finished training model and fitting best hyperparameters for LogisticRegression. F1 score: 0.7697724864859653
Fitting 5 folds for each of 100 candidates, totalling 500 fits


/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

Finished training model and fitting best hyperparameters for RandomForestClassifier. F1 score: 0.5959807046687382
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Finished training model and fitting best hyperparameters for GradientBoostingClassifier. F1 score: 0.6839559011521602
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Finished training model and fitting best hyperparameters for XGBClassifier. F1 score: 0.6955424995534666
Fitting 5 folds for each of 100 candidates, totalling 500 fits


...

Finished training model and fitting best hyperparameters for KNeighborsClassifier. F1 score: 0.6532536329237979
Fitting 5 folds for each of 100 candidates, totalling 500 fits


/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

Finished training model and fitting best hyperparameters for SGDClassifier. F1 score: 0.6919085276506618
Fitting 5 folds for each of 100 candidates, totalling 500 fits


/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valerio/workspace/caltech/sentence_

Finished training model and fitting best hyperparameters for Perceptron. F1 score: 0.7038705406451942
precision: 0.7030629174746822 ± 0.22093934542309468
recall: 0.9154035087719299 ± 0.040154886338917097
fscore: 0.7697724864859653 ± 0.15235849332395549
selected model: LogisticRegression
selected params {'C': 0.005496168690813228, 'class_weight': 'balanced', 'solver': 'liblinear', 'warm_start': True}


## Task 3

### Expression

In [9]:
y = [int(row[4]) for row in expression_unified_ds]
classifier, precision, recall, fscore, classifier_name, params, std_precision, std_recall, std_f1 = train_classifier(corpus_expr, y)
joblib.dump(classifier, '../classifiers/language_expression.joblib')
print(f'precision: {str(precision)} ± {str(std_precision)}')
print(f'recall: {str(recall)} ± {str(std_recall)}')
print(f'fscore: {str(fscore)} ± {str(std_f1)}')
print(f'selected model: {classifier_name}')
print(f'selected params {str(params)}')

Fitting 5 folds for each of 100 candidates, totalling 500 fits


/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

Finished training model and fitting best hyperparameters for LogisticRegression. F1 score: 0.8814822887694611
Fitting 5 folds for each of 100 candidates, totalling 500 fits


/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/scipy/optimize/_linesearch.py:306: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valerio/workspac

Finished training model and fitting best hyperparameters for RandomForestClassifier. F1 score: 0.8330425603171792
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Finished training model and fitting best hyperparameters for GradientBoostingClassifier. F1 score: 0.8643771709979664
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Finished training model and fitting best hyperparameters for XGBClassifier. F1 score: 0.8687973577390824
Fitting 5 folds for each of 100 candidates, totalling 500 fits


...

Finished training model and fitting best hyperparameters for KNeighborsClassifier. F1 score: 0.8253599583582736
Fitting 5 folds for each of 100 candidates, totalling 500 fits


/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

Finished training model and fitting best hyperparameters for SGDClassifier. F1 score: 0.8833709797668551
Fitting 5 folds for each of 100 candidates, totalling 500 fits


/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/linear_

Finished training model and fitting best hyperparameters for Perceptron. F1 score: 0.8781101978719628
precision: 0.9430952188137344 ± 0.030503470430255322
recall: 0.8553731830801894 ± 0.14693473229102094
fscore: 0.8876530454457997 ± 0.08326927628290504
selected model: SVC
selected params {'C': 44.469363517319884, 'class_weight': None, 'coef0': 0.43440089238170054, 'decision_function_shape': 'ovo', 'degree': 4, 'gamma': 0.002413812560675785, 'kernel': 'sigmoid', 'random_state': 49, 'shrinking': False, 'tol': 0.005979247861812558}


### Kinaseact

In [10]:
y = [int(row[4]) for row in kinaseact_unified_ds]
classifier, precision, recall, fscore, classifier_name, params, std_precision, std_recall, std_f1 = train_classifier(corpus_kinaseact, y)
joblib.dump(classifier, '../classifiers/language_kinase.joblib')
print(f'precision: {str(precision)} ± {str(std_precision)}')
print(f'recall: {str(recall)} ± {str(std_recall)}')
print(f'fscore: {str(fscore)} ± {str(std_f1)}')
print(f'selected model: {classifier_name}')
print(f'selected params {str(params)}')

Fitting 5 folds for each of 100 candidates, totalling 500 fits


/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_divisio

Finished training model and fitting best hyperparameters for LogisticRegression. F1 score: 0.9460613864902513
Fitting 5 folds for each of 100 candidates, totalling 500 fits


/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/met

Finished training model and fitting best hyperparameters for RandomForestClassifier. F1 score: 0.8898171501119517
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Finished training model and fitting best hyperparameters for GradientBoostingClassifier. F1 score: 0.9339929664394818
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Finished training model and fitting best hyperparameters for XGBClassifier. F1 score: 0.9328470434928798
Fitting 5 folds for each of 100 candidates, totalling 500 fits


...

Finished training model and fitting best hyperparameters for KNeighborsClassifier. F1 score: 0.8771804939311789
Fitting 5 folds for each of 100 candidates, totalling 500 fits


/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/scipy/optimize/_linesearch.py:306: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/utils/o

Finished training model and fitting best hyperparameters for SGDClassifier. F1 score: 0.9501060954393583
Fitting 5 folds for each of 100 candidates, totalling 500 fits


/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valerio/workspace/caltech/sentence_classification/venv/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/home/valerio/workspace/caltech/sentenc

Finished training model and fitting best hyperparameters for Perceptron. F1 score: 0.932401730514749
precision: 0.9657903403645243 ± 0.021959319125923426
recall: 0.9410002278423331 ± 0.054567339769251835
fscore: 0.9518693600574677 ± 0.023919375156594865
selected model: SVC
selected params {'C': 42.932130470132115, 'class_weight': 'balanced', 'coef0': 0.1057989791671804, 'decision_function_shape': 'ovo', 'degree': 8, 'gamma': 0.0015885697209272686, 'kernel': 'rbf', 'random_state': 44, 'shrinking': True, 'tol': 0.007405671832254371}
